<h1>TFM: CREANDO OBJETO SCREENPRO2<h1>

<h3>Ajustar formato de la librería<h3>

In [ ]:
import pandas as pd

# Cargar el archivo CSV sin nombres asignados
library_path = "./TFM_12O/TFM_previo/library.csv"
library = pd.read_csv(library_path, header=None)  # Cargar sin encabezado (header=None)

# Asignar nombres a las columnas
library.columns = ['target', 'sequence', 'sgID']

# Verificar los cambios
print("Columnas renombradas:")
print(library.head())

# Guardar el archivo con las columnas renombradas
output_path = "./TFM_12O/TFM_previo/library_renamed.csv"
library.to_csv(output_path, index=False)

print(f"Archivo con columnas renombradas guardado en: {output_path}")


<h1>GENERAR COUNTS CON SCREENPRO2<h1>

!pip install numba==0.56.4

```python
%pip install ScreenPro2
%pip install polars
%pip install anndata
%pip install biobear
%pip install simple_colors
%pip install pydeseq2
%pip install scanpy
%pip install adjustText
```


```python
#Generación de counts con el script fastq_to_counts.py
import sys
import os

#Agregar el directorio './TFM_12O/code' al sys.path
code_dir = os.path.abspath("./TFM_12O/code")
if code_dir not in sys.path:
    sys.path.append(code_dir)

patuS_dir = "./TFM_12O/01.QC/PatuS_reads"
patuT_dir = "./TFM_12O/01.QC/PatuT_reads"

def extract_samples(directory):
    """
    Extrae los nombres completos de los archivos FASTQ, incluyendo la extensión .fastq.
    """
    sample_ids = []
    for file_name in os.listdir(directory):
        if file_name.endswith(".fastq.gz"):  # Procesar solo archivos FASTQ
            sample_ids.append(file_name.split(".fastq.gz")[0])  # Eliminar solo la extensión
    return sample_ids

#Extraer muestras de ambas carpetas
patuS_samples = extract_samples(patuS_dir)
patuT_samples = extract_samples(patuT_dir)

#Importar el módulo
from fastqgz_to_counts import parallelSeqFileToCountsParallel
import multiprocessing

fasta_output_dir = "./TFM_12O/02.Counts/PatuT_reads/unaligned_fasta"
count_output_dir = "./TFM_12O/02.Counts/PatuT_reads/counts"

#Generar listas para fastq_files, fasta_outputs y count_outputs con rutas completas
fastq_files = [os.path.join(patuT_dir, f"{sample}.fastq.gz") for sample in patuT_samples]
fasta_outputs = [os.path.join(fasta_output_dir, f"{sample}_unaligned.fa") for sample in patuT_samples]
count_outputs = [os.path.join(count_output_dir, f"{sample}.counts") for sample in patuT_samples]

#Archivo FASTA de la biblioteca
library_fasta = "./TFM_12O/TFM_previo/library_renamed.fasta"

#Parámetros adicionales
start_index = None  # Índice inicial para recortar (opcional, None si no recortas)
stop_index = None   # Índice final para recortar (opcional, None si no recortas)
test_mode = False   # Cambiar a True para modo de prueba

#Crear un pool de procesos para procesamiento paralelo
pool = multiprocessing.Pool(processes=4)  # Cambia "4" al número de núcleos disponibles

#Llamar a la función para procesar en paralelo
results = parallelSeqFileToCountsParallel(
    fastqGzFileNameList=fastq_files,
    fastaFileNameList=fasta_outputs,
    countFileNameList=count_outputs,
    libraryFasta=library_fasta,
    processPool=pool,
    startIndex=start_index,
    stopIndex=stop_index,
    test=test_mode
)

#Cerrar el pool de procesos
pool.close()
pool.join()

#Mostrar resultados
for result in results:
    print("Archivo procesado:", result[0])
    print("Total de lecturas procesadas:", result[1][0])
    print("Total de lecturas alineadas:", result[1][1])
    print("Porcentaje de alineación:", result[1][2], "%")

```

```python
#Hay una lectura que era un singletone, lo rellenamos con 0 el R2 porque los R2 tienen alrededor de un 0,3% de mapeo, así que el error es mínimo.
#Por lo visto, se desordenaron los sgRNAs en el archivo de conteos, así que vamos a reordenarlos según su R1, el resto deberian estar bien.
import pandas as pd

# Ruta del archivo problemático y del archivo de referencia
archivo_problematico = r"C:\Users\alvar\Desktop\TFM 12O\TFM_12O\02.Counts\scp2\PatuS_reads\counts\PatuS-Cas9-14d-4_R2_trimmed.counts"
archivo_referencia = r"C:\Users\alvar\Desktop\TFM 12O\TFM_12O\02.Counts\scp2\PatuS_reads\counts\PatuS-14d-1_R1_trimmed.counts"

# Leer referencia para obtener el orden correcto de sgRNAs
sgRNAs_ref = pd.read_csv(archivo_referencia, sep=None, engine='python', header=None, usecols=[0])[0].tolist()

# Leer archivo problemático
df_prob = pd.read_csv(archivo_problematico, sep=None, engine='python', header=None, names=['sgRNA', 'count'])

# Reordenar según la referencia
df_prob_reordenado = df_prob.set_index('sgRNA').reindex(sgRNAs_ref).reset_index()

# Sobreescribir el archivo original
df_prob_reordenado.to_csv(archivo_problematico, sep='\t', header=False, index=False)

# Mostrar las primeras filas para comprobar
print(df_prob_reordenado.head())
```

In [ ]:
# Escribir la matriz de counts ordenada para PatuS y guardarla en un TSV

import os
import pandas as pd

# Rutas
counts_dir = r"C:\Users\alvar\Desktop\TFM 12O\TFM_12O\02.Counts\scp2\PatuS_reads\counts"
output_tsv = r"C:\Users\alvar\Desktop\TFM 12O\csv2\PatuS_X.tsv"
library_path = r"./TFM_12O/TFM_previo/library.csv"

orden_muestras = [
    # PatuS-7d
    "PatuS-7d-1", "PatuS-7d-2", "PatuS-7d-3", "PatuS-7d-4", "PatuS-7d-5",
    # PatuS-14d
    "PatuS-14d-1", "PatuS-14d-2", "PatuS-14d-3", "PatuS-14d-4", "PatuS-14d-5",
    # PatuS-21d
    "PatuS-21d-1", "PatuS-21d-2", "PatuS-21d-3", "PatuS-21d-4", "PatuS-21d-5",
    # PatuS-Cas9-7d
    "PatuS-Cas9-7d-1", "PatuS-Cas9-7d-2", "PatuS-Cas9-7d-3", "PatuS-Cas9-7d-4", "PatuS-Cas9-7d-5",
    # PatuS-Cas9-14d
    "PatuS-Cas9-14d-1", "PatuS-Cas9-14d-2", "PatuS-Cas9-14d-3", "PatuS-Cas9-14d-4", "PatuS-Cas9-14d-5",
    # PatuS-Cas9-21d
    "PatuS-Cas9-21d-1", "PatuS-Cas9-21d-2", "PatuS-Cas9-21d-3", "PatuS-Cas9-21d-4", "PatuS-Cas9-21d-5",
    # pEPIGEN
    "pEPIGEN1", "pEPIGEN2"
]

# Leer los sgRNAs del primer archivo R1 para fijar el orden inicial
first_file = os.path.join(counts_dir, f"{orden_muestras[0]}_R1_trimmed.counts")
sgRNAs = pd.read_csv(first_file, sep=None, engine='python', header=None, usecols=[0])[0].tolist()
print("Orden de muestras (lecturas):", orden_muestras)
print("Número de sgRNAs:", len(sgRNAs))

all_counts = []

for sample in orden_muestras:
    r1_file = f"{sample}_R1_trimmed.counts"
    r2_file = f"{sample}_R2_trimmed.counts"
    r1_path = os.path.join(counts_dir, r1_file)
    r2_path = os.path.join(counts_dir, r2_file)
    df_r1 = pd.read_csv(r1_path, sep=None, engine='python', header=None, names=['sgRNA', 'count'])
    df_r2 = pd.read_csv(r2_path, sep=None, engine='python', header=None, names=['sgRNA', 'count'])
    df_r1 = df_r1.set_index('sgRNA').reindex(sgRNAs).fillna(0)
    df_r2 = df_r2.set_index('sgRNA').reindex(sgRNAs).fillna(0)
    summed = df_r1['count'].astype(int) + df_r2['count'].astype(int)
    all_counts.append(summed.values)

counts_df = pd.DataFrame(all_counts, columns=sgRNAs, index=orden_muestras)

# --- REORDENAR sgRNAs SEGÚN LA LIBRERÍA DE REFERENCIA (ROBUSTO) ---
library_df = pd.read_csv(library_path, header=None)
# Limpiar nombres de sgRNAs en ambos lados
sgRNA_order = [str(x).strip().replace('\ufeff', '') for x in library_df[0].tolist()]
counts_df.columns = [str(x).strip().replace('\ufeff', '') for x in counts_df.columns]

# Solo mantener y reordenar los sgRNAs que están en ambos
sgRNAs_final = [sg for sg in sgRNA_order if sg in counts_df.columns]
counts_df = counts_df[sgRNAs_final]

# Guardar CSV con nombres de sgRNAs como encabezado y muestras como primera columna
counts_df.index.name = "sample"
counts_df.to_csv(output_tsv, sep='\t')

print("\nControl final:")
print("Primeros sgRNAs (columnas):", list(counts_df.columns[:5]))
print("Primeras muestras (filas):", list(counts_df.index[:5]))
print("Shape:", counts_df.shape)
print(f"Guardado: {output_tsv}")

In [ ]:
# Escribir la matriz de counts ordenada para PatuT y guardarla en un TSV

import os
import pandas as pd

# Rutas
counts_dir_t = r"C:\Users\alvar\Desktop\TFM 12O\TFM_12O\02.Counts\scp2\PatuT_reads\counts"
output_tsv_t = r"C:\Users\alvar\Desktop\TFM 12O\csv2\PatuT_X.tsv"
library_path_t = r"./TFM_12O/TFM_previo/library.csv"

orden_muestras_t = [
    # PatuT-7d
    "PatuT-7d-1", "PatuT-7d-2", "PatuT-7d-3", "PatuT-7d-4", "PatuT-7d-5",
    # PatuT-14d
    "PatuT-14d-1", "PatuT-14d-2", "PatuT-14d-3", "PatuT-14d-4", "PatuT-14d-5",
    # PatuT-21d
    "PatuT-21d-1", "PatuT-21d-2", "PatuT-21d-3", "PatuT-21d-4", "PatuT-21d-5",
    # PatuT-Cas9-7d
    "PatuT-Cas9-7d-1", "PatuT-Cas9-7d-2", "PatuT-Cas9-7d-3", "PatuT-Cas9-7d-4", "PatuT-Cas9-7d-5",
    # PatuT-Cas9-14d
    "PatuT-Cas9-14d-1", "PatuT-Cas9-14d-2", "PatuT-Cas9-14d-3", "PatuT-Cas9-14d-4", "PatuT-Cas9-14d-5",
    # PatuT-Cas9-21d
    "PatuT-Cas9-21d-1", "PatuT-Cas9-21d-2", "PatuT-Cas9-21d-3", "PatuT-Cas9-21d-4", "PatuT-Cas9-21d-5",
    # pEPIGEN
    "pEPIGEN1", "pEPIGEN2"
]

# Leer los sgRNAs del primer archivo R1 para fijar el orden
first_file_t = os.path.join(counts_dir_t, f"{orden_muestras_t[0]}_R1_trimmed.counts")
sgRNAs_t = pd.read_csv(first_file_t, sep=None, engine='python', header=None, usecols=[0])[0].tolist()
print("Orden de muestras (lecturas):", orden_muestras_t)
print("Número de sgRNAs:", len(sgRNAs_t))

all_counts_t = []

for sample in orden_muestras_t:
    r1_file = f"{sample}_R1_trimmed.counts"
    r2_file = f"{sample}_R2_trimmed.counts"
    r1_path = os.path.join(counts_dir_t, r1_file)
    r2_path = os.path.join(counts_dir_t, r2_file)
    df_r1 = pd.read_csv(r1_path, sep=None, engine='python', header=None, names=['sgRNA', 'count'])
    df_r2 = pd.read_csv(r2_path, sep=None, engine='python', header=None, names=['sgRNA', 'count'])
    df_r1 = df_r1.set_index('sgRNA').reindex(sgRNAs_t).fillna(0)
    df_r2 = df_r2.set_index('sgRNA').reindex(sgRNAs_t).fillna(0)
    summed = df_r1['count'].astype(int) + df_r2['count'].astype(int)
    all_counts_t.append(summed.values)

counts_df_t = pd.DataFrame(all_counts_t, columns=sgRNAs_t, index=orden_muestras_t)

# --- REORDENAR sgRNAs SEGÚN LA LIBRERÍA DE REFERENCIA (ROBUSTO) ---
library_df_t = pd.read_csv(library_path_t, header=None)
# Usar la PRIMERA columna de la librería para el orden de sgRNAs
sgRNA_order_t = [str(x).strip().replace('\ufeff', '') for x in library_df_t[0].tolist()]
counts_df_t.columns = [str(x).strip().replace('\ufeff', '') for x in counts_df_t.columns]

# Solo mantener y reordenar los sgRNAs que están en ambos
sgRNAs_final_t = [sg for sg in sgRNA_order_t if sg in counts_df_t.columns]
counts_df_t = counts_df_t[sgRNAs_final_t]

# Guardar CSV con nombres de sgRNAs como encabezado y muestras como primera columna
counts_df_t.index.name = "sample"
counts_df_t.to_csv(output_tsv_t, sep='\t')

print("\nControl final:")
print("Primeros sgRNAs (columnas):", list(counts_df_t.columns[:5]))
print("Primeras muestras (filas):", list(counts_df_t.index[:5]))
print("Shape:", counts_df_t.shape)
print(f"Guardado: {output_tsv_t}")

<h1>GENERAR OBJETO ANNDATA <h1>

<H5> GENERAR ANNDATA.X <H5>

In [ ]:
#Generar el anndata.X para PatuS y PatuT

import pandas as pd
import numpy as np

# Leer la matriz de counts de PatuS (sin encabezados)
counts_path_patuS = r"C:\Users\alvar\Desktop\TFM 12O\csv2\PatuS_X.tsv"
counts_df_patuS = pd.read_csv(counts_path_patuS, sep='\t', header=0, index_col=0)
patuS_X = counts_df_patuS.values  # numpy array

# Leer la matriz de counts de PatuT (sin encabezados)
counts_path_patuT = r"C:\Users\alvar\Desktop\TFM 12O\csv2\PatuT_X.tsv"
counts_df_patuT = pd.read_csv(counts_path_patuT, sep='\t', header=0, index_col=0)
patuT_X = counts_df_patuT.values  # numpy array

print("Shape matriz PatuS:", patuS_X.shape)
print("Shape matriz PatuT:", patuT_X.shape)

In [ ]:
#Generar el anndata.X para PatuS y PatuT con counts provenientes de mageckcount.
import pandas as pd
import numpy as np

# Leer la librería y obtener el orden correcto de sgRNAs (primera columna)
library_path = "./TFM_12O/TFM_previo/library.csv"
library_df = pd.read_csv(library_path, header=None)
sgRNA_order = [str(x).strip().replace('\ufeff', '') for x in library_df[0].tolist()]

# PatuS
file_path_s = r"TFM_12O\02.Counts\mageckcount\all_samples_PatuS.count.txt"
df_s = pd.read_csv(file_path_s, sep='\t')
df_s['sgRNA'] = df_s['sgRNA'].astype(str).str.strip().str.replace('\ufeff', '', regex=False)
df_s = df_s.set_index('sgRNA')
sgRNAs_final = [sg for sg in sgRNA_order if sg in df_s.index]
df_s = df_s.loc[sgRNAs_final]
print("Primeros sgRNAs PatuS (ordenados):", list(df_s.index[:5]))
patuS_mageck_X = df_s.iloc[:, 1:].T.to_numpy()  # Omitir columna 'Gene'
print("Shape matriz PatuS (muestras, sgRNAs):", patuS_mageck_X.shape)

# PatuT
file_path_t = r"TFM_12O\02.Counts\mageckcount\all_samples_PatuT.count.txt"
df_t = pd.read_csv(file_path_t, sep='\t')
df_t['sgRNA'] = df_t['sgRNA'].astype(str).str.strip().str.replace('\ufeff', '', regex=False)
df_t = df_t.set_index('sgRNA')
sgRNAs_final_t = [sg for sg in sgRNA_order if sg in df_t.index]
df_t = df_t.loc[sgRNAs_final_t]
print("Primeros sgRNAs PatuT (ordenados):", list(df_t.index[:5]))
patuT_mageck_X = df_t.iloc[:, 1:].T.to_numpy()
print("Shape matriz PatuT (muestras, sgRNAs):", patuT_mageck_X.shape)


In [ ]:
#Generar el anndata.X para PatuS con counts corregidos por CNV provenientes del script fastq_to_counts.py.
import pandas as pd
import numpy as np

# Definir el orden esperado de las muestras y sgRNAs
orden_muestras_t = [
    "PatuS-7d-1", "PatuS-7d-2", "PatuS-7d-3", "PatuS-7d-4", "PatuS-7d-5",
    "PatuS-14d-1", "PatuS-14d-2", "PatuS-14d-3", "PatuS-14d-4", "PatuS-14d-5",
    "PatuS-21d-1", "PatuS-21d-2", "PatuS-21d-3", "PatuS-21d-4", "PatuS-21d-5",
    "PatuS-Cas9-7d-1", "PatuS-Cas9-7d-2", "PatuS-Cas9-7d-3", "PatuS-Cas9-7d-4", "PatuS-Cas9-7d-5",
    "PatuS-Cas9-14d-1", "PatuS-Cas9-14d-2", "PatuS-Cas9-14d-3", "PatuS-Cas9-14d-4", "PatuS-Cas9-14d-5",
    "PatuS-Cas9-21d-1", "PatuS-Cas9-21d-2", "PatuS-Cas9-21d-3", "PatuS-Cas9-21d-4", "PatuS-Cas9-21d-5",
    "pEPIGEN1", "pEPIGEN2"
]

# Cargar el archivo de referencia de sgRNAs
library = pd.read_csv("./TFM_12O/TFM_previo/library_renamed.csv")
orden_sgrnas = library['target'].astype(str).tolist()

# Leer el archivo de counts asegurando que todo se carga como string
counts = pd.read_csv("csv2/PatuS_CNVcorrected.tsv", sep='\t', dtype=str)

# Establecer el índice por sgRNA y asegurar la presencia de los samples esperados
counts = counts.set_index('sgRNA')
counts = counts.loc[:, orden_muestras_t]  # Filtrar solo las columnas correctas

# Asegurar el orden de los sgRNAs
counts = counts.reindex(orden_sgrnas)

# Puntos de control para confirmar orden correcto
print("Chequeo de orden de samples:")
print("Esperado:", orden_muestras_t)
print("En la matriz:", list(counts.columns))
print("¿Coinciden?", orden_muestras_t == list(counts.columns))
print()

print("Chequeo de orden de sgRNAs:")
print("Primeros 5 esperados:", orden_sgrnas[:5])
print("Primeros 5 en la matriz:", list(counts.index[:5]))
print("Últimos 5 esperados:", orden_sgrnas[-5:])
print("Últimos 5 en la matriz:", list(counts.index[-5:]))
print("¿Coinciden?", orden_sgrnas == list(counts.index))
print()

# Convertir a numpy array eliminando encabezados y descriptoras
counts = counts.astype(float).astype(int)  # Redondear y pasar a int
PatuS_CNV_X = np.transpose(counts.values)  # Transponer para obtener el shape correcto (32, 1574)
pd.DataFrame(PatuS_CNV_X).to_csv("csv2/PatuS_CNV_X.tsv", sep='\t', index=False, header=False, quoting=3)

# Puntos de control finales
print("Shape de PatuS_CNV_X:", PatuS_CNV_X.shape)  # Esperado (32, 1574)
print("Primeros valores:\n", PatuS_CNV_X[:5, :5])  # Verificación visual

In [ ]:
#Generar el anndata.X para PatuT con counts corregidos por CNV provenientes del script fastq_to_counts.py.
import pandas as pd
import numpy as np

# Definir el orden esperado de las muestras y sgRNAs
orden_muestras_t = [
    "PatuT-7d-1", "PatuT-7d-2", "PatuT-7d-3", "PatuT-7d-4", "PatuT-7d-5",
    "PatuT-14d-1", "PatuT-14d-2", "PatuT-14d-3", "PatuT-14d-4", "PatuT-14d-5",
    "PatuT-21d-1", "PatuT-21d-2", "PatuT-21d-3", "PatuT-21d-4", "PatuT-21d-5",
    "PatuT-Cas9-7d-1", "PatuT-Cas9-7d-2", "PatuT-Cas9-7d-3", "PatuT-Cas9-7d-4", "PatuT-Cas9-7d-5",
    "PatuT-Cas9-14d-1", "PatuT-Cas9-14d-2", "PatuT-Cas9-14d-3", "PatuT-Cas9-14d-4", "PatuT-Cas9-14d-5",
    "PatuT-Cas9-21d-1", "PatuT-Cas9-21d-2", "PatuT-Cas9-21d-3", "PatuT-Cas9-21d-4", "PatuT-Cas9-21d-5",
    "pEPIGEN1", "pEPIGEN2"
]

# Cargar el archivo de referencia de sgRNAs
library = pd.read_csv("./TFM_12O/TFM_previo/library_renamed.csv")
orden_sgrnas = library['target'].astype(str).tolist()

# Leer el archivo de counts asegurando que todo se carga como string
counts = pd.read_csv("csv2/PatuT_CNVcorrected.tsv", sep='\t', dtype=str)

# Establecer el índice por sgRNA y asegurar la presencia de los samples esperados
counts = counts.set_index('sgRNA')
counts = counts.loc[:, orden_muestras_t]  # Filtrar solo las columnas correctas

# Asegurar el orden de los sgRNAs
counts = counts.reindex(orden_sgrnas)

# Puntos de control para confirmar orden correcto
print("Chequeo de orden de samples:")
print("Esperado:", orden_muestras_t)
print("En la matriz:", list(counts.columns))
print("¿Coinciden?", orden_muestras_t == list(counts.columns))
print()

print("Chequeo de orden de sgRNAs:")
print("Primeros 5 esperados:", orden_sgrnas[:5])
print("Primeros 5 en la matriz:", list(counts.index[:5]))
print("Últimos 5 esperados:", orden_sgrnas[-5:])
print("Últimos 5 en la matriz:", list(counts.index[-5:]))
print("¿Coinciden?", orden_sgrnas == list(counts.index))
print()

# Convertir a numpy array eliminando encabezados y descriptoras
counts = counts.astype(float).astype(int)  # Redondear y pasar a int
PatuT_CNV_X = np.transpose(counts.values)  # Transponer para obtener el shape correcto (32, 1574)
pd.DataFrame(PatuT_CNV_X).to_csv("csv2/PatuT_CNV_X.tsv", sep='\t', index=False, header=False, quoting=3)

# Puntos de control finales
print("Shape de PatuT_CNV_X:", PatuT_CNV_X.shape)  # Esperado (32, 1574)
print("Primeros valores:\n", PatuT_CNV_X[:5, :5])  # Verificación visual

In [ ]:
#Generar el anndata.X para PatuS con counts corregidos por CNV provenientes de mageckcount.
import pandas as pd
import numpy as np

# Definir el orden esperado de las muestras y sgRNAs para PatuS
orden_muestras_s = [
    "PatuS-7d-1", "PatuS-7d-2", "PatuS-7d-3", "PatuS-7d-4", "PatuS-7d-5",
    "PatuS-14d-1", "PatuS-14d-2", "PatuS-14d-3", "PatuS-14d-4", "PatuS-14d-5",
    "PatuS-21d-1", "PatuS-21d-2", "PatuS-21d-3", "PatuS-21d-4", "PatuS-21d-5",
    "PatuS-Cas9-7d-1", "PatuS-Cas9-7d-2", "PatuS-Cas9-7d-3", "PatuS-Cas9-7d-4", "PatuS-Cas9-7d-5",
    "PatuS-Cas9-14d-1", "PatuS-Cas9-14d-2", "PatuS-Cas9-14d-3", "PatuS-Cas9-14d-4", "PatuS-Cas9-14d-5",
    "PatuS-Cas9-21d-1", "PatuS-Cas9-21d-2", "PatuS-Cas9-21d-3", "PatuS-Cas9-21d-4", "PatuS-Cas9-21d-5",
    "pEPIGEN1", "pEPIGEN2"
]

# Cargar el archivo de referencia de sgRNAs
library = pd.read_csv("./TFM_12O/TFM_previo/library_renamed.csv")
orden_sgrnas = library['target'].astype(str).tolist()

# Leer el archivo de counts asegurando que todo se carga como string
counts = pd.read_csv("csv2/PatuS_mageck_CNVcorrected.tsv", sep='\t', dtype=str)

# Limpiar nombres de sgRNAs y columnas
counts['sgRNA'] = counts['sgRNA'].astype(str).str.strip().replace('\ufeff', '', regex=True)
counts.columns = [str(col).strip().replace('\ufeff', '') for col in counts.columns]

# Establecer el índice por sgRNA
counts = counts.set_index('sgRNA')

# Solo mantener muestras y sgRNAs presentes en ambos lados (sin NAs)
samples_final = [s for s in orden_muestras_s if s in counts.columns]
sgRNAs_final = [sg for sg in orden_sgrnas if sg in counts.index]

counts = counts.loc[sgRNAs_final, samples_final]

# Puntos de control para confirmar orden correcto
print("Chequeo de orden de samples:")
print("Esperado:", samples_final)
print("En la matriz:", list(counts.columns))
print("¿Coinciden?", samples_final == list(counts.columns))
print()

print("Chequeo de orden de sgRNAs:")
print("Primeros 5 esperados:", sgRNAs_final[:5])
print("Primeros 5 en la matriz:", list(counts.index[:5]))
print("Últimos 5 esperados:", sgRNAs_final[-5:])
print("Últimos 5 en la matriz:", list(counts.index[-5:]))
print("¿Coinciden?", sgRNAs_final == list(counts.index))
print()

# Convertir a numpy array eliminando encabezados y descriptoras
counts = counts.astype(float).astype(int)  # Redondear y pasar a int
PatuS_mageck_CNV_X = np.transpose(counts.values)  # (n_samples, n_sgRNAs)
pd.DataFrame(PatuS_mageck_CNV_X).to_csv("csv2/PatuS_mageck_CNV_X.tsv", sep='\t', index=False, header=False, quoting=3)

# Puntos de control finales
print("Shape de PatuS_mageck_CNV_X:", PatuS_mageck_CNV_X.shape)
print("Primeros valores:\n", PatuS_mageck_CNV_X[:5, :5])

In [ ]:
#Generar el anndata.X para PatuT con counts corregidos por CNV provenientes de mageckcount.
import pandas as pd
import numpy as np

# Definir el orden esperado de las muestras y sgRNAs
orden_muestras_t = [
    "PatuT-7d-1", "PatuT-7d-2", "PatuT-7d-3", "PatuT-7d-4", "PatuT-7d-5",
    "PatuT-14d-1", "PatuT-14d-2", "PatuT-14d-3", "PatuT-14d-4", "PatuT-14d-5",
    "PatuT-21d-1", "PatuT-21d-2", "PatuT-21d-3", "PatuT-21d-4", "PatuT-21d-5",
    "PatuT-Cas9-7d-1", "PatuT-Cas9-7d-2", "PatuT-Cas9-7d-3", "PatuT-Cas9-7d-4", "PatuT-Cas9-7d-5",
    "PatuT-Cas9-14d-1", "PatuT-Cas9-14d-2", "PatuT-Cas9-14d-3", "PatuT-Cas9-14d-4", "PatuT-Cas9-14d-5",
    "PatuT-Cas9-21d-1", "PatuT-Cas9-21d-2", "PatuT-Cas9-21d-3", "PatuT-Cas9-21d-4", "PatuT-Cas9-21d-5",
    "pEPIGEN1", "pEPIGEN2"
]

# Cargar el archivo de referencia de sgRNAs
library = pd.read_csv("./TFM_12O/TFM_previo/library_renamed.csv")
orden_sgrnas = library['target'].astype(str).tolist()

# Leer el archivo de counts asegurando que todo se carga como string
counts = pd.read_csv("csv2/PatuT_mageck_CNVcorrected.tsv", sep='\t', dtype=str)

# Limpiar nombres de sgRNAs y columnas
counts['sgRNA'] = counts['sgRNA'].astype(str).str.strip().replace('\ufeff', '', regex=True)
counts.columns = [str(col).strip().replace('\ufeff', '') for col in counts.columns]

# Establecer el índice por sgRNA
counts = counts.set_index('sgRNA')

# Solo mantener muestras y sgRNAs presentes en ambos lados (sin NAs)
samples_final = [s for s in orden_muestras_t if s in counts.columns]
sgRNAs_final = [sg for sg in orden_sgrnas if sg in counts.index]

counts = counts.loc[sgRNAs_final, samples_final]

# Puntos de control para confirmar orden correcto
print("Chequeo de orden de samples:")
print("Esperado:", samples_final)
print("En la matriz:", list(counts.columns))
print("¿Coinciden?", samples_final == list(counts.columns))
print()

print("Chequeo de orden de sgRNAs:")
print("Primeros 5 esperados:", sgRNAs_final[:5])
print("Primeros 5 en la matriz:", list(counts.index[:5]))
print("Últimos 5 esperados:", sgRNAs_final[-5:])
print("Últimos 5 en la matriz:", list(counts.index[-5:]))
print("¿Coinciden?", sgRNAs_final == list(counts.index))
print()

# Convertir a numpy array eliminando encabezados y descriptoras
counts = counts.astype(float).astype(int)  # Redondear y pasar a int
PatuT_mageck_CNV_X = np.transpose(counts.values)  # (n_samples, n_sgRNAs)
pd.DataFrame(PatuT_mageck_CNV_X).to_csv("csv2/PatuT_mageck_CNV_X.tsv", sep='\t', index=False, header=False, quoting=3)

# Puntos de control finales
print("Shape de PatuT_mageck_CNV_X:", PatuT_mageck_CNV_X.shape)
print("Primeros valores:\n", PatuT_mageck_CNV_X[:5, :5])

<H5> GENERAR ANNDATA.OBS <H5>

In [ ]:
import pandas as pd

# Lista de muestras finales 
final_samples = [
    "PatuS-7d-1", "PatuS-7d-2", "PatuS-7d-3", "PatuS-7d-4", "PatuS-7d-5",
    "PatuS-14d-1", "PatuS-14d-2", "PatuS-14d-3", "PatuS-14d-4", "PatuS-14d-5",
    "PatuS-21d-1", "PatuS-21d-2", "PatuS-21d-3", "PatuS-21d-4", "PatuS-21d-5",
    "PatuS-Cas9-7d-1", "PatuS-Cas9-7d-2", "PatuS-Cas9-7d-3", "PatuS-Cas9-7d-4", "PatuS-Cas9-7d-5",
    "PatuS-Cas9-14d-1", "PatuS-Cas9-14d-2", "PatuS-Cas9-14d-3", "PatuS-Cas9-14d-4", "PatuS-Cas9-14d-5",
    "PatuS-Cas9-21d-1", "PatuS-Cas9-21d-2", "PatuS-Cas9-21d-3", "PatuS-Cas9-21d-4", "PatuS-Cas9-21d-5",
    "pEPIGEN1", "pEPIGEN2"
]

# Crear DataFrame meta solo con las muestras finales
meta_df_patuS = pd.DataFrame({"sample_name": final_samples})

# Asignar la columna 'condition' según tu criterio
def assign_condition(name):
    if "Cas9" in name:
        return "treated"
    else:
        return "control"

meta_df_patuS["condition"] = meta_df_patuS["sample_name"].apply(assign_condition)

# Extraer el número de replicado
def extract_replicate(sample_name):
    parts = sample_name.split("-")
    if "Cas9" in parts:  # Caso de muestras con 'Cas9'
        return int(parts[3].split("_")[0])
    elif "pEPIGEN" in sample_name:  # Caso de controles 'pEPIGEN'
        return int(sample_name.split("pEPIGEN")[1].split("_")[0])
    else:  # Otros casos
        return int(parts[2].split("_")[0])

meta_df_patuS["replicate"] = meta_df_patuS["sample_name"].apply(extract_replicate)

print(meta_df_patuS)
print("Shape de la matriz de counts:", meta_df_patuS.shape)

In [ ]:
import pandas as pd

# Lista de muestras finales 
final_samples_patuT = [
    "PatuT-7d-1", "PatuT-7d-2", "PatuT-7d-3", "PatuT-7d-4", "PatuT-7d-5",
    "PatuT-14d-1", "PatuT-14d-2", "PatuT-14d-3", "PatuT-14d-4", "PatuT-14d-5",
    "PatuT-21d-1", "PatuT-21d-2", "PatuT-21d-3", "PatuT-21d-4", "PatuT-21d-5",
    "PatuT-Cas9-7d-1", "PatuT-Cas9-7d-2", "PatuT-Cas9-7d-3", "PatuT-Cas9-7d-4", "PatuT-Cas9-7d-5",
    "PatuT-Cas9-14d-1", "PatuT-Cas9-14d-2", "PatuT-Cas9-14d-3", "PatuT-Cas9-14d-4", "PatuT-Cas9-14d-5",
    "PatuT-Cas9-21d-1", "PatuT-Cas9-21d-2", "PatuT-Cas9-21d-3", "PatuT-Cas9-21d-4", "PatuT-Cas9-21d-5",
    "pEPIGEN1", "pEPIGEN2"
]

# Crear DataFrame meta solo con las muestras finales de PatuT
meta_df_patuT = pd.DataFrame({"sample_name": final_samples_patuT})

# Asignar la columna 'condition' según tu criterio
def assign_condition(name):
    if "Cas9" in name:
        return "treated"
    else:
        return "control"

meta_df_patuT["condition"] = meta_df_patuT["sample_name"].apply(assign_condition)

# Extraer el número de replicado
def extract_replicate(sample_name):
    parts = sample_name.split("-")
    if "Cas9" in parts:  # Caso de muestras con 'Cas9'
        return int(parts[3].split("_")[0])
    elif "pEPIGEN" in sample_name:  # Caso de controles 'pEPIGEN'
        return int(sample_name.split("pEPIGEN")[1].split("_")[0])
    else:  # Otros casos
        return int(parts[2].split("_")[0])

meta_df_patuT["replicate"] = meta_df_patuT["sample_name"].apply(extract_replicate)

print(meta_df_patuT)
print("Shape de la matriz de counts:", meta_df_patuT.shape)

<H5> GENERAR EL ANNDATA.VAR <H5>

In [ ]:
#Generar el df target_df_patuS
import pandas as pd

# Cargar la librería
library_renamed_path = "./TFM_12O/TFM_previo/library_renamed.csv"
library_df = pd.read_csv(library_renamed_path)

# Fusionar sgID y target en una sola columna "target" con formato "sgID -> target"
library_df["target"] = library_df["sgID"].astype(str) + " -> " + library_df["target"].astype(str)

# Leer el archivo de controles negativos (un target por línea, sin cabecera)
neg_ctrl_path = "./TFM_12O/TFM_previo/PatuS_neg_controls_guides.txt"
with open(neg_ctrl_path, "r") as f:
    negative_controls = set(line.strip() for line in f if line.strip())

# Asignar targetType: "negative_control" si el target original está en la lista, si no "experimental"
library_df["targetType"] = library_df["target"].apply(
    lambda x: "negative_control" if x.split(" -> ")[1] in negative_controls else "experimental"
)

# Crear el DataFrame final solo con las columnas necesarias
target_df_patuS = library_df[["target", "targetType"]].copy()

# Guardar el DataFrame target_df como un archivo .tsv en la carpeta csv2

output_target_tsv = r"C:\Users\alvar\Desktop\TFM 12O\csv2\target_df_patuS.tsv"
target_df_patuS.to_csv(output_target_tsv, sep='\t', index=False)

print(f"Archivo guardado en: {output_target_tsv}")
print("Shape de target_df:", target_df_patuS.shape)
print("Primeros 5 targets:", target_df_patuS.head())

In [ ]:
#Generar el df target_df_patuT
import pandas as pd

# Cargar la librería
library_renamed_path = "./TFM_12O/TFM_previo/library_renamed.csv"
library_df = pd.read_csv(library_renamed_path)

# Fusionar sgID y target en una sola columna "target" con formato "sgID -> target"
library_df["target"] = library_df["sgID"].astype(str) + " -> " + library_df["target"].astype(str)

# Leer el archivo de controles negativos (un target por línea, sin cabecera)
neg_ctrl_path = "./TFM_12O/TFM_previo/PatuT_neg_controls_guides.txt"
with open(neg_ctrl_path, "r") as f:
    negative_controls = set(line.strip() for line in f if line.strip())

# Asignar targetType: "negative_control" si el target original está en la lista, si no "experimental"
library_df["targetType"] = library_df["target"].apply(
    lambda x: "negative_control" if x.split(" -> ")[1] in negative_controls else "experimental"
)

# Crear el DataFrame final solo con las columnas necesarias
target_df_patuT = library_df[["target", "targetType"]].copy()

# Guardar el DataFrame target_df como un archivo .tsv en la carpeta csv2

output_target_tsv = r"C:\Users\alvar\Desktop\TFM 12O\csv2\target_df_patuT.tsv"
target_df_patuT.to_csv(output_target_tsv, sep='\t', index=False)

print(f"Archivo guardado en: {output_target_tsv}")
print("Shape de target_df:", target_df_patuT.shape)
)

<H5> CREACIÓN DEL OBJETO ANNDATA <H5>

In [ ]:
#PatuS + Counts fastq_to_counts.py
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuS_adata = ad.AnnData(
    X   = patuS_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuS,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuS  # pandas dataframe of targets metadata including "target" and "targetType" columns
)


In [ ]:
#PatuT + Counts fastq_to_counts.py
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuT_adata = ad.AnnData(
    X   = patuT_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuT,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuT  # pandas dataframe of targets metadata including "target" and "targetType" columns
)


In [ ]:
#PatuS + Counts mageckcount
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuS_mageck_adata = ad.AnnData(
    X   = patuS_mageck_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuS,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuS  # pandas dataframe of targets metadata including "target" and "targetType" columns
)


In [ ]:
#patuT + Counts mageckcount
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuT_mageck_adata = ad.AnnData(
    X   = patuT_mageck_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuT,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuT  # pandas dataframe of targets metadata including "target" and "targetType" columns
)


In [ ]:
#PatuS + counts fastq_to_counts.py corregidos por CNV
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuS_CNV_adata = ad.AnnData(
    X   = PatuS_CNV_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuS,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuS  # pandas dataframe of targets metadata including "target" and "targetType" columns
)

In [ ]:
#PatuT + counts fastq_to_counts.py corregidos por CNV
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuT_CNV_adata = ad.AnnData(
    X   = PatuT_CNV_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuT,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuT  # pandas dataframe of targets metadata including "target" and "targetType" columns
)

In [ ]:
#PatuS + counts mageckcount corregidos por CNV
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuS_mageck_CNV_adata = ad.AnnData(
    X   = PatuS_mageck_CNV_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuS,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuS  # pandas dataframe of targets metadata including "target" and "targetType" columns
)

In [ ]:
#PatuT + counts mageckcount corregidos por CNV
import pandas as pd
import anndata as ad
from screenpro.assays import PooledScreens

patuT_mageck_CNV_adata = ad.AnnData(
    X   = PatuT_mageck_CNV_X, # pandas dataframe of counts (samples x targets)
    obs = meta_df_patuT,   # pandas dataframe of samples metadata including "condition" and "replicate" columns
    var = target_df_patuT  # pandas dataframe of targets metadata including "target" and "targetType" columns
)

<H3> CÁLCULO DEL FENOTIPO <H3>

<H5> PATUS CORREGIDO POR CNV ANNDATA<H5>

In [ ]:
from screenpro.phenoscore import runPhenoScore

result_name, result_df_patuS_CNV = runPhenoScore(
    adata=patuS_CNV_adata,
    cond_ref="control",
    cond_test="treated",
    score_level="compare_reps",
    var_names="target",
    test="ttest",
    ctrl_label="negative_control"
)

# Mostrar los primeros resultados obtenidos
print(f"Resultados guardados bajo el nombre: {result_name}")
print(result_df_patuS_CNV.head())

# Filtrar e imprimir todos los resultados significativos (BH adj_pvalue <= 0.05)
significativos = result_df_patuS_CNV[result_df_patuS_CNV["BH adj_pvalue"] <= 0.05]
print(f"Resultados significativos (BH adj_pvalue <= 0.05): {len(significativos)}")
print(significativos)

# Volcano plot para result_df_patuS (score vs. -log10(ttest pvalue)), resaltando sgRNAs con |score| ≥ 0.15 y ttest pvalue < 0.05

import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

df = result_df_patuS_CNV.copy()
df['-log10_ttest_pvalue'] = -np.log10(df['ttest pvalue'])
is_significant = (df['ttest pvalue'] < 0.05) & (np.abs(df['score']) >= 0.15)

plt.figure(figsize=(9, 7))
plt.scatter(df['score'], df['-log10_ttest_pvalue'], c='grey', alpha=0.5, label='Not significant')
plt.scatter(df[is_significant]['score'], df[is_significant]['-log10_ttest_pvalue'],
            c='red', alpha=0.8, label='Significant (|score| ≥ 0.15 & ttest pvalue < 0.05)')

# Etiquetas bonitas con adjustText
texts = []
for _, row in df[is_significant].iterrows():
    texts.append(
        plt.text(
            row['score'], row['-log10_ttest_pvalue'],
            str(row['target']),
            fontsize=10, color='black', weight='bold',
            bbox=dict(facecolor='white', edgecolor='red', boxstyle='round,pad=0.2'),
            ha='left', va='center'
        )
    )

# Ajustar las etiquetas para que no se salgan ni se solapen
adjust_text(
    texts,
    only_move={'points':'y', 'texts':'y'},  # Solo mover en vertical
    arrowprops=dict(arrowstyle='-', color='red', lw=1)
)

plt.axvline(x=0.15, color='blue', linestyle='--', linewidth=1)
plt.axvline(x=-0.15, color='blue', linestyle='--', linewidth=1)
plt.axhline(y=-np.log10(0.05), color='green', linestyle='--', linewidth=1)

plt.xlabel('Phenotype Score')
plt.ylabel('-log10(pvalue)')
plt.title('Volcano plot PatuS')
plt.legend()
plt.tight_layout()
plt.show()

# Ranked phenotype score plot for PatuS corrected by CNV
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

# 1. Prepare data
df = result_df_patuS_CNV.copy()
df = df.dropna(subset=['score', 'BH adj_pvalue'])
df['gene'] = df['target'].str.split('--?>').str[0].str.strip()
df['-log10_pval'] = -np.log10(np.clip(df['BH adj_pvalue'], 1e-300, 1))

# 2. Sort by score for global ranking
df = df.sort_values('score')
df = df.reset_index(drop=True)
df['rank'] = np.arange(1, len(df)+1)

# 3. Find lowest and highest score for each gene
idx_min = df.groupby('gene')['score'].idxmin()
idx_max = df.groupby('gene')['score'].idxmax()
extremos = pd.concat([df.loc[idx_min], df.loc[idx_max]]).drop_duplicates()

# 4. Select 10 depleted genes and 10 enriched genes
top_depleted = extremos.nsmallest(10, 'score')  # 10 depleted
top_enriched = extremos.nlargest(10, 'score')   # 10 enriched
idx_to_label = set(top_depleted.index).union(set(top_enriched.index))

# 5. Color and size
colors = np.where(df['score'] < 0, '#1f77b4', '#d62728')
sizes = np.interp(df['-log10_pval'], (df['-log10_pval'].min(), df['-log10_pval'].max()), (40, 250))

# 6. Plot
plt.figure(figsize=(10, 7))
plt.scatter(df['rank'], df['score'], c=colors, s=sizes, alpha=0.85, edgecolor='k', linewidth=0.5, zorder=3)

# 7. BALANCED LABELS - With sufficient separation to avoid overlaps
# Separate depleted and enriched genes
depleted_genes = []
enriched_genes = []

for idx in idx_to_label:
    row = df.loc[idx]
    if row['score'] < 0:
        depleted_genes.append((idx, row))
    else:
        enriched_genes.append((idx, row))

# Sort for better distribution
depleted_genes.sort(key=lambda x: x[1]['score'])
# Enriched genes will be sorted in the labeling section to avoid crossings

# Labels for DEPLETED genes (to the RIGHT of each point)
for i, (idx, row) in enumerate(depleted_genes):
    point_x = row['rank']
    point_y = row['score']
    
    # Label position: 6% of width to the right + slight vertical offset
    label_x = point_x + len(df) * 0.06  # 6% of total width
    label_y = point_y + (i - len(depleted_genes)/2) * 0.05  # Vertical separation
    
    plt.annotate(row['gene'], 
                xy=(point_x, point_y),  # Point where it points
                xytext=(label_x, label_y),  # Text position
                fontsize=9, fontweight='bold', color='#1f77b4',
                ha='left', va='center',
                bbox=dict(boxstyle='round,pad=0.25', 
                         facecolor='white', 
                         edgecolor='#1f77b4', 
                         alpha=0.85,
                         linewidth=1),
                arrowprops=dict(arrowstyle='->', 
                               color='#1f77b4', 
                               lw=1.0, 
                               alpha=0.7,
                               shrinkA=3,    # More space from text
                               shrinkB=8),   # Distance to point
                zorder=10)

# Labels for ENRICHED genes (to the LEFT of each point)
# Sort by descending score so highest labels are on top and arrows don't cross
enriched_genes_sorted = sorted(enriched_genes, key=lambda x: x[1]['score'], reverse=True)

for i, (idx, row) in enumerate(enriched_genes_sorted):
    point_x = row['rank']
    point_y = row['score']
    
    # Label position: LONGER ARROWS - 12% of width to the left + progressive vertical separation
    label_x = point_x - len(df) * 0.12  # 12% of total width (longer arrows)
    
    # Progressive vertical separation: highest labels go higher
    base_y = max([r[1]['score'] for r in enriched_genes]) + 0.1  # Base from highest score
    label_y = base_y - (i * 0.08)  # Vertical separation of 0.08 between labels
    
    plt.annotate(row['gene'], 
                xy=(point_x, point_y),  # Point where it points
                xytext=(label_x, label_y),  # Text position
                fontsize=9, fontweight='bold', color='#d62728',
                ha='right', va='center',
                bbox=dict(boxstyle='round,pad=0.25', 
                         facecolor='white', 
                         edgecolor='#d62728', 
                         alpha=0.85,
                         linewidth=1),
                arrowprops=dict(arrowstyle='->', 
                               color='#d62728', 
                               lw=1.2,      # Slightly thicker arrows
                               alpha=0.8,   # Slightly more opaque
                               shrinkA=3,   # Space from text
                               shrinkB=8),  # Distance to point
                zorder=10)

# 8. Final aesthetics
plt.axhline(0, color='black', linewidth=0.8, zorder=1)
plt.xticks([])
plt.yticks(fontsize=10)
plt.xlabel('')
plt.ylabel('Phenotype score', fontsize=12)
plt.title('Ranked phenotype score plot PatuS', fontsize=14, pad=10)

# 9. Gene color legend (positioned to not overlap the plot)
from matplotlib.patches import Patch
color_legend = [
    Patch(facecolor='#d62728', alpha=0.8, label='Enriched genes (red)'),
    Patch(facecolor='#1f77b4', alpha=0.8, label='Depleted genes (blue)')
]

# Color legend at the top
legend1 = plt.legend(handles=color_legend, 
                    loc='upper center', 
                    bbox_to_anchor=(0.5, 0.95),
                    ncol=2,
                    frameon=True, 
                    fancybox=True, 
                    shadow=True,
                    fontsize=11)

# 10. Point size legend (p-value) in the bottom right corner
legend_elements_pval = []
for pval in [0.05, 0.01, 0.001]:
    size = np.interp(-np.log10(pval), (df['-log10_pval'].min(), df['-log10_pval'].max()), (40, 250))
    legend_elements_pval.append(plt.scatter([], [], c='gray', alpha=0.5, s=size, label=f'p = {pval}'))

legend2 = plt.legend(handles=legend_elements_pval,
                    scatterpoints=1, 
                    frameon=False, 
                    labelspacing=1, 
                    title='P-value', 
                    loc='lower right')

# Add both legends
plt.gca().add_artist(legend1)

plt.tight_layout()
plt.show()

# Statistics
print(f"\n=== BALANCED PLOT ===")
print(f"🔴 Enriched genes: {len(enriched_genes)} (left labels with long arrows)")
print(f"🔵 Depleted genes: {len(depleted_genes)} (right labels)")
print(f"✨ Long arrows and ordered labels to avoid crossings and overlaps")
print(f"📊 Total points: {len(df)}")

print(f"\nEnriched genes (red - left labels, ordered without crossings):")
for _, (idx, row) in enumerate(enriched_genes_sorted):
    print(f"  🔴 {row['gene']}: {row['score']:.3f}")

print(f"\nDepleted genes (blue - right labels):")
for _, (idx, row) in enumerate(depleted_genes):
    print(f"  🔵 {row['gene']}: {row['score']:.3f}")

<H5> PATUT CORREGIDO POR CNV ANNDATA<H5>

In [ ]:
from screenpro.phenoscore import runPhenoScore

result_name, result_df_patuT_CNV = runPhenoScore(
    adata=patuT_CNV_adata,
    cond_ref="control",
    cond_test="treated",
    score_level="compare_reps",
    var_names="target",
    test="ttest",
    ctrl_label="negative_control"
)

# Mostrar los primeros resultados obtenidos
print(f"Resultados guardados bajo el nombre: {result_name}")
print(result_df_patuT_CNV.head())

# Filtrar e imprimir todos los resultados significativos (BH adj_pvalue <= 0.05)
significativos = result_df_patuT_CNV[result_df_patuT_CNV["BH adj_pvalue"] <= 0.05]
print(f"Resultados significativos (BH adj_pvalue <= 0.05): {len(significativos)}")
print(significativos)

# Volcano plot para result_df_patuS (score vs. -log10(BH adj_pvalue)), resaltando sgRNAs con |score| ≥ 0.15 y BH adj_pvalue < 0.05

import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

df = result_df_patuT_CNV.copy()
df['-log10_BH_adj_pvalue'] = -np.log10(df['BH adj_pvalue'])
is_significant = (df['BH adj_pvalue'] < 0.05) & (np.abs(df['score']) >= 0.15)

plt.figure(figsize=(9, 7))
plt.scatter(df['score'], df['-log10_BH_adj_pvalue'], c='grey', alpha=0.5, label='Not significant')
plt.scatter(df[is_significant]['score'], df[is_significant]['-log10_BH_adj_pvalue'],
            c='red', alpha=0.8, label='Significant (|score| ≥ 0.15 & BH adj_pvalue < 0.05)')

# Etiquetas bonitas con adjustText
texts = []
for _, row in df[is_significant].iterrows():
    texts.append(
        plt.text(
            row['score'], row['-log10_BH_adj_pvalue'],
            str(row['target']),
            fontsize=10, color='black', weight='bold',
            bbox=dict(facecolor='white', edgecolor='red', boxstyle='round,pad=0.2'),
            ha='left', va='center'
        )
    )

# Ajustar las etiquetas para que no se salgan ni se solapen
adjust_text(
    texts,
    only_move={'points':'y', 'texts':'y'},  # Solo mover en vertical
    arrowprops=dict(arrowstyle='-', color='red', lw=1)
)

plt.axvline(x=0.15, color='blue', linestyle='--', linewidth=1)
plt.axvline(x=-0.15, color='blue', linestyle='--', linewidth=1)
plt.axhline(y=-np.log10(0.05), color='green', linestyle='--', linewidth=1)

plt.xlabel('Score')
plt.ylabel('-log10(BH adj_pvalue)')
plt.title('Volcano plot PatuT')
plt.legend()
plt.tight_layout()
plt.show()

# Ranked phenotype score plot for PatuT corrected by CNV
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

# 1. Prepare data
df = result_df_patuT_CNV.copy()
df = df.dropna(subset=['score', 'BH adj_pvalue'])
df['gene'] = df['target'].str.split('--?>').str[0].str.strip()
df['-log10_pval'] = -np.log10(np.clip(df['BH adj_pvalue'], 1e-300, 1))

# 2. Sort by score for global ranking
df = df.sort_values('score')
df = df.reset_index(drop=True)
df['rank'] = np.arange(1, len(df)+1)

# 3. Find lowest and highest score for each gene
idx_min = df.groupby('gene')['score'].idxmin()
idx_max = df.groupby('gene')['score'].idxmax()
extremos = pd.concat([df.loc[idx_min], df.loc[idx_max]]).drop_duplicates()

# 4. Select 10 depleted genes and 6 enriched genes
top_depleted = extremos.nsmallest(10, 'score')  # 10 depleted
top_enriched = extremos.nlargest(6, 'score')   # 6 enriched
idx_to_label = set(top_depleted.index).union(set(top_enriched.index))

# 5. Color and size
colors = np.where(df['score'] < 0, '#1f77b4', '#d62728')
sizes = np.interp(df['-log10_pval'], (df['-log10_pval'].min(), df['-log10_pval'].max()), (40, 250))

# 6. Plot
plt.figure(figsize=(10, 7))
plt.scatter(df['rank'], df['score'], c=colors, s=sizes, alpha=0.85, edgecolor='k', linewidth=0.5, zorder=3)

# 7. BALANCED LABELS - With sufficient separation to avoid overlaps
# Separate depleted and enriched genes
depleted_genes = []
enriched_genes = []

for idx in idx_to_label:
    row = df.loc[idx]
    if row['score'] < 0:
        depleted_genes.append((idx, row))
    else:
        enriched_genes.append((idx, row))

# Sort for better distribution
depleted_genes.sort(key=lambda x: x[1]['score'])
# Enriched genes will be sorted in the labeling section to avoid crossings

# Labels for DEPLETED genes (to the RIGHT of each point)
for i, (idx, row) in enumerate(depleted_genes):
    point_x = row['rank']
    point_y = row['score']
    
    # Label position: 6% of width to the right + slight vertical offset
    label_x = point_x + len(df) * 0.06  # 6% of total width
    label_y = point_y + (i - len(depleted_genes)/2) * 0.05  # Vertical separation
    
    plt.annotate(row['gene'], 
                xy=(point_x, point_y),  # Point where it points
                xytext=(label_x, label_y),  # Text position
                fontsize=9, fontweight='bold', color='#1f77b4',
                ha='left', va='center',
                bbox=dict(boxstyle='round,pad=0.25', 
                         facecolor='white', 
                         edgecolor='#1f77b4', 
                         alpha=0.85,
                         linewidth=1),
                arrowprops=dict(arrowstyle='->', 
                               color='#1f77b4', 
                               lw=1.0, 
                               alpha=0.7,
                               shrinkA=3,    # More space from text
                               shrinkB=8),   # Distance to point
                zorder=10)

# Labels for ENRICHED genes (to the LEFT of each point)
# Sort by descending score so highest labels are on top and arrows don't cross
enriched_genes_sorted = sorted(enriched_genes, key=lambda x: x[1]['score'], reverse=True)

for i, (idx, row) in enumerate(enriched_genes_sorted):
    point_x = row['rank']
    point_y = row['score']
    
    # Label position: LONGER ARROWS - 12% of width to the left + progressive vertical separation
    label_x = point_x - len(df) * 0.12  # 12% of total width (longer arrows)
    
    # Progressive vertical separation: highest labels go higher
    base_y = max([r[1]['score'] for r in enriched_genes]) + 0.1  # Base from highest score
    label_y = base_y - (i * 0.08)  # Vertical separation of 0.08 between labels
    
    plt.annotate(row['gene'], 
                xy=(point_x, point_y),  # Point where it points
                xytext=(label_x, label_y),  # Text position
                fontsize=9, fontweight='bold', color='#d62728',
                ha='right', va='center',
                bbox=dict(boxstyle='round,pad=0.25', 
                         facecolor='white', 
                         edgecolor='#d62728', 
                         alpha=0.85,
                         linewidth=1),
                arrowprops=dict(arrowstyle='->', 
                               color='#d62728', 
                               lw=1.2,      # Slightly thicker arrows
                               alpha=0.8,   # Slightly more opaque
                               shrinkA=3,   # Space from text
                               shrinkB=8),  # Distance to point
                zorder=10)

# 8. Final aesthetics
plt.axhline(0, color='black', linewidth=0.8, zorder=1)
plt.xticks([])
plt.yticks(fontsize=10)
plt.xlabel('')
plt.ylabel('Phenotype score', fontsize=12)
plt.title('Ranked phenotype score plot PatuT', fontsize=14, pad=10)

# 9. Gene color legend (positioned to not overlap the plot)
from matplotlib.patches import Patch
color_legend = [
    Patch(facecolor='#d62728', alpha=0.8, label='Enriched genes (red)'),
    Patch(facecolor='#1f77b4', alpha=0.8, label='Depleted genes (blue)')
]

# Color legend at the top
legend1 = plt.legend(handles=color_legend, 
                    loc='upper center', 
                    bbox_to_anchor=(0.5, 0.95),
                    ncol=2,
                    frameon=True, 
                    fancybox=True, 
                    shadow=True,
                    fontsize=11)

# 10. Point size legend (p-value) in the bottom right corner
legend_elements_pval = []
for pval in [0.05, 0.01, 0.001]:
    size = np.interp(-np.log10(pval), (df['-log10_pval'].min(), df['-log10_pval'].max()), (40, 250))
    legend_elements_pval.append(plt.scatter([], [], c='gray', alpha=0.5, s=size, label=f'p = {pval}'))

legend2 = plt.legend(handles=legend_elements_pval,
                    scatterpoints=1, 
                    frameon=False, 
                    labelspacing=1, 
                    title='P-value', 
                    loc='lower right')

# Add both legends
plt.gca().add_artist(legend1)

plt.tight_layout()
plt.show()

# Statistics
print(f"\n=== BALANCED PLOT ===")
print(f"🔴 Enriched genes: {len(enriched_genes)} (left labels with long arrows)")
print(f"🔵 Depleted genes: {len(depleted_genes)} (right labels)")
print(f"✨ Long arrows and ordered labels to avoid crossings and overlaps")
print(f"📊 Total points: {len(df)}")

print(f"\nEnriched genes (red - left labels, ordered without crossings):")
for _, (idx, row) in enumerate(enriched_genes_sorted):
    print(f"  🔴 {row['gene']}: {row['score']:.3f}")

print(f"\nDepleted genes (blue - right labels):")
for _, (idx, row) in enumerate(depleted_genes):
    print(f"  🔵 {row['gene']}: {row['score']:.3f}")

<H5> PATUS mageckcount CORREGIDO POR CNV ANNDATA<H5>

In [ ]:
from screenpro.phenoscore import runPhenoScore

result_name, result_df_patuS_mageck_CNV = runPhenoScore(
    adata=patuS_mageck_CNV_adata,
    cond_ref="control",
    cond_test="treated",
    score_level="compare_reps",
    var_names="target",
    test="ttest",
    ctrl_label="negative_control"
)

# Mostrar los primeros resultados obtenidos
print(f"Resultados guardados bajo el nombre: {result_name}")
print(result_df_patuS_mageck_CNV.head())

# Filtrar e imprimir todos los resultados significativos (BH adj_pvalue <= 0.05)
significativos = result_df_patuS_mageck_CNV[result_df_patuS_mageck_CNV["BH adj_pvalue"] <= 0.05]
print(f"Resultados significativos (BH adj_pvalue <= 0.05): {len(significativos)}")
print(significativos)

# Volcano plot para result_df_patuS (score vs. -log10(ttest pvalue)), resaltando sgRNAs con |score| ≥ 0.15 y ttest pvalue < 0.05

import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

df = result_df_patuS_mageck_CNV.copy()
df['-log10_ttest_pvalue'] = -np.log10(df['ttest pvalue'])
is_significant = (df['ttest pvalue'] < 0.05) & (np.abs(df['score']) >= 0.15)

plt.figure(figsize=(9, 7))
plt.scatter(df['score'], df['-log10_ttest_pvalue'], c='grey', alpha=0.5, label='No significativo')
plt.scatter(df[is_significant]['score'], df[is_significant]['-log10_ttest_pvalue'],
            c='red', alpha=0.8, label='Significativo (|score| ≥ 0.15 y ttest pvalue < 0.05)')

# Etiquetas bonitas con adjustText
texts = []
for _, row in df[is_significant].iterrows():
    texts.append(
        plt.text(
            row['score'], row['-log10_ttest_pvalue'],
            str(row['target']),
            fontsize=10, color='black', weight='bold',
            bbox=dict(facecolor='white', edgecolor='red', boxstyle='round,pad=0.2'),
            ha='left', va='center'
        )
    )

# Ajustar las etiquetas para que no se salgan ni se solapen
adjust_text(
    texts,
    only_move={'points':'y', 'texts':'y'},  # Solo mover en vertical
    arrowprops=dict(arrowstyle='-', color='red', lw=1)
)

plt.axvline(x=0.15, color='blue', linestyle='--', linewidth=1)
plt.axvline(x=-0.15, color='blue', linestyle='--', linewidth=1)
plt.axhline(y=-np.log10(0.05), color='green', linestyle='--', linewidth=1)

plt.xlabel('Score')
plt.ylabel('-log10(ttest pvalue)')
plt.title('Volcano plot: Score vs. Significancia por sgRNA (PatuS mageckcount corregido por CNV)')
plt.legend()
plt.tight_layout()
plt.show()

<H5> PATUT mageckcount CORREGIDO POR CNV ANNDATA<H5>

In [ ]:
from screenpro.phenoscore import runPhenoScore

result_name, result_df_patuT_mageck_CNV = runPhenoScore(
    adata=patuT_mageck_CNV_adata,
    cond_ref="control",
    cond_test="treated",
    score_level="compare_reps",
    var_names="target",
    test="ttest",
    ctrl_label="negative_control"
)

# Mostrar los primeros resultados obtenidos
print(f"Resultados guardados bajo el nombre: {result_name}")
print(result_df_patuT_mageck_CNV.head())

# Filtrar e imprimir todos los resultados significativos (BH adj_pvalue <= 0.05)
significativos = result_df_patuT_mageck_CNV[result_df_patuT_mageck_CNV["BH adj_pvalue"] <= 0.05]
print(f"Resultados significativos (BH adj_pvalue <= 0.05): {len(significativos)}")
print(significativos)

# Volcano plot para result_df_patuS (score vs. -log10(BH adj_pvalue)), resaltando sgRNAs con |score| ≥ 0.15 y BH adj_pvalue < 0.05

import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text

df = result_df_patuT_mageck_CNV.copy()
df['-log10_BH_adj_pvalue'] = -np.log10(df['BH adj_pvalue'])
is_significant = (df['BH adj_pvalue'] < 0.05) & (np.abs(df['score']) >= 0.15)

plt.figure(figsize=(9, 7))
plt.scatter(df['score'], df['-log10_BH_adj_pvalue'], c='grey', alpha=0.5, label='No significativo')
plt.scatter(df[is_significant]['score'], df[is_significant]['-log10_BH_adj_pvalue'],
            c='red', alpha=0.8, label='Significativo (|score| ≥ 0.15 y BH adj_pvalue < 0.05)')

# Etiquetas bonitas con adjustText
texts = []
for _, row in df[is_significant].iterrows():
    texts.append(
        plt.text(
            row['score'], row['-log10_BH_adj_pvalue'],
            str(row['target']),
            fontsize=10, color='black', weight='bold',
            bbox=dict(facecolor='white', edgecolor='red', boxstyle='round,pad=0.2'),
            ha='left', va='center'
        )
    )

# Ajustar las etiquetas para que no se salgan ni se solapen
adjust_text(
    texts,
    only_move={'points':'y', 'texts':'y'},  # Solo mover en vertical
    arrowprops=dict(arrowstyle='-', color='red', lw=1)
)

plt.axvline(x=0.15, color='blue', linestyle='--', linewidth=1)
plt.axvline(x=-0.15, color='blue', linestyle='--', linewidth=1)
plt.axhline(y=-np.log10(0.05), color='green', linestyle='--', linewidth=1)

plt.xlabel('Score')
plt.ylabel('-log10(BH adj_pvalue)')
plt.title('Volcano plot: Score vs. Significancia por sgRNA (PatuS mageckcount corregido por CNV)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#Comparación de counts entre los generados por el script #fastq_to_counts.py y los generados por mageckcount

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar counts de tu script (muestras en filas, sgRNAs en columnas)
counts_script = pd.read_csv(r"C:\Users\alvar\Desktop\TFM 12O\csv2\PatuS_X.tsv", sep='\t', index_col=0)

# Cargar counts de mageck (sgRNAs en filas, muestras en columnas)
counts_mageck = pd.read_csv(r"TFM_12O\02.Counts\mageckcount\all_samples_PatuS.count.txt", sep='\t', index_col=0)
counts_mageck_T = counts_mageck.T  # Trasponer para que las muestras sean filas

# Seleccionar la primera muestra (ajusta el nombre si es necesario)
sample_name = counts_script.index[20]
first_sample_script = counts_script.loc[sample_name]
first_sample_mageck = counts_mageck_T.loc[sample_name]

# Limpiar índices de posibles espacios, BOM y duplicados
first_sample_script.index = first_sample_script.index.astype(str).str.strip().str.replace('\ufeff', '', regex=False)
first_sample_mageck.index = first_sample_mageck.index.astype(str).str.strip().str.replace('\ufeff', '', regex=False)

# Si hay duplicados, sumarlos
first_sample_script = first_sample_script.groupby(first_sample_script.index).sum()
first_sample_mageck = first_sample_mageck.groupby(first_sample_mageck.index).sum()

# Alinear los sgRNAs por nombre
df_compare = pd.DataFrame({
    'count_script': first_sample_script,
    'count_mageck': first_sample_mageck
}).fillna(0)

# Graficar en escala log-log
plt.figure(figsize=(7, 7))
plt.scatter(df_compare['count_script'] + 1, df_compare['count_mageck'] + 1, alpha=0.6)
plt.plot([1, df_compare['count_script'].max()+1], [1, df_compare['count_script'].max()+1], 'k--', alpha=0.7)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Counts (Script fastq_to_counts.py)')
plt.ylabel('Counts (MAGeCK)')
plt.title('Comparación de counts por sgRNA')
plt.tight_layout()
plt.show()

# Mostrar sgRNAs con mayores diferencias absolutas
df_compare['abs_diff'] = np.abs(df_compare['count_script'] - df_compare['count_mageck'])
top_diff = df_compare.sort_values('abs_diff', ascending=False).head(10)
print("sgRNAs con mayores diferencias absolutas:")
print(top_diff)

![image.png](attachment:image.png)

Referencia 2: Inactivation of the Euchromatic Histone-Lysine N-Methyltransferase G9a Attenuates Oncogenic KRAS-Driven Pancreatic Cancer (PMC8261250)

Referencia 4: Smarcd3 is an epigenetic modulator of the metabolic landscape in pancreatic cancer (PMC9849267)

In [ ]:
# Analysis of EHMT2 and PRMT7 sgRNAs in PatuT CNV corrected data
# Show score and statistical significance for all sgRNAs targeting EHMT2 and PRMT7

import pandas as pd

# Filter sgRNAs containing EHMT2 or PRMT7 in their target name
ehmt2_prmt7_sgrnas = result_df_patuT_CNV[
    result_df_patuT_CNV['target'].str.contains('EHMT2|PRMT7', case=False, na=False)
].copy()

# Select relevant columns: target, score, ttest pvalue, BH adj_pvalue
ehmt2_prmt7_results = ehmt2_prmt7_sgrnas[['target', 'score', 'ttest pvalue', 'BH adj_pvalue']].copy()

# Sort by score (from most depleted to most enriched)
ehmt2_prmt7_results = ehmt2_prmt7_results.sort_values('score')

print("=== EHMT2 and PRMT7 sgRNAs Analysis ===")
print(f"Total sgRNAs found: {len(ehmt2_prmt7_results)}")
print("\nResults (sorted by score):")
print(ehmt2_prmt7_results.to_string(index=False))

# Summary statistics
print(f"\n=== Summary ===")
print(f"EHMT2 sgRNAs: {len(ehmt2_prmt7_results[ehmt2_prmt7_results['target'].str.contains('EHMT2', case=False)])}")
print(f"PRMT7 sgRNAs: {len(ehmt2_prmt7_results[ehmt2_prmt7_results['target'].str.contains('PRMT7', case=False)])}")

# Significant sgRNAs (ttest pvalue < 0.05)
significant_ttest = ehmt2_prmt7_results[ehmt2_prmt7_results['ttest pvalue'] < 0.05]
print(f"Significant by ttest pvalue (< 0.05): {len(significant_ttest)}")

# Significant sgRNAs (BH adj_pvalue < 0.05)
significant_bh = ehmt2_prmt7_results[ehmt2_prmt7_results['BH adj_pvalue'] < 0.05]
print(f"Significant by BH adj_pvalue (< 0.05): {len(significant_bh)}")

if len(significant_ttest) > 0:
    print(f"\nSignificant sgRNAs (ttest pvalue < 0.05):")
    print(significant_ttest.to_string(index=False))

if len(significant_bh) > 0:
    print(f"\nSignificant sgRNAs (BH adj_pvalue < 0.05):")
    print(significant_bh.to_string(index=False))